# Modelling metabolism in soy mature leaves

### Generating SOY core model

In [1]:
from cobra import io
from cobra.core import Metabolite
from cobra import flux_analysis

#import sbml file
fname = "Data/Soy_core_model_GA.sbml"
model = io.sbml.read_sbml_model(fname)

#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
met.compartment="b"
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
rxn.add_metabolites({met:1})

In [2]:
len(model.genes)

2098

### Importing transcript data

In [3]:
import pandas as pd
df = pd.read_csv("/home/sanu/Downloads/gmax_leaf_CO2_normExprn_1.txt",sep="\t")

/home/sanu/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051 DtypeWarning: Columns (0,2,3,13,14) have mixed types. Specify dtype option on import or set low_memory=False.


In [4]:
amb_list = list(df.columns[0:17])
ele_list = list(df.columns[0:17])
for x in df.columns:
    if "Amb" in x:
        amb_list.append(x)
    elif "Ele" in x:
        ele_list.append(x)
    else:
        continue

print(amb_list)
print(ele_list)

['Glyma2.0', 'Query_Length', 'Uniport_ID', 'Hit_Alias', 'Hit_Length', 'E-Value', 'Alignment_Length', 'Query_Coverage', 'Percent_Identity', 'Identities_Covered_percent', 'Number_of_Matches', 'Bit_Score', 'HSPs', 'gene name/function', 'EC', 'ProbeID', 'Glyma 1.1', 'X474_E_Amb.CEL', 'X554_E_Amb.CEL', 'X560_E_Amb.CEL', 'X561_E_Amb.CEL', 'X580_E_Amb.CEL', 'X582_E_Amb.CEL', 'X583_E_Amb.CEL', 'X585_E_Amb.CEL']
['Glyma2.0', 'Query_Length', 'Uniport_ID', 'Hit_Alias', 'Hit_Length', 'E-Value', 'Alignment_Length', 'Query_Coverage', 'Percent_Identity', 'Identities_Covered_percent', 'Number_of_Matches', 'Bit_Score', 'HSPs', 'gene name/function', 'EC', 'ProbeID', 'Glyma 1.1', 'X460_E_Ele.CEL', 'X467_E_Ele.CEL', 'X468_E_Ele.CEL', 'X469_E_Ele.CEL', 'X556_E_Ele.CEL', 'X557_E_Ele.CEL', 'X558_E_Ele.CEL', 'X559_E_Ele.CEL']


In [5]:
df_amb = df[amb_list]
df_ele = df[ele_list]

In [6]:
modelGenes = dict()
for gene in model.genes:
    geneID = gene.id.split(".")[0]+"."+gene.id.split(".")[1]
    modelGenes[geneID]=gene


In [7]:
modelGenesNotInTranscript = set(modelGenes.keys()) - set(df["Glyma2.0"])

In [8]:
len(modelGenesNotInTranscript)

1094

In [9]:
fout = open("/home/sanu/SoyGenesInModel.csv","w")
for gene in model.genes:
    geneID = gene.id.split(".")[0]+"."+gene.id.split(".")[1]
    fout.write(geneID+"\n")
fout.close()

In [10]:
fout = open("/home/sanu/SoyGenesInModelMinusTranscriptomics.csv","w")
for gene in modelGenesNotInTranscript:
    geneID = gene.split(".")[0]+"."+gene.split(".")[1]
    fout.write(geneID+"\n")
fout.close()

In [12]:
i=0
for rxn in model.reactions:
    if len(rxn.genes)!=0:
        i=i+1
print(i)

401


### Use biomass composition to generate biomass equations

In [13]:
import pandas as pd

df_biomass = pd.read_csv("Data/biomass_soy.csv")


from cobra.core import Reaction

rxn = Reaction("Biomass_leaf_tx")
for met in df_biomass["Unnamed: 0"]:
    rxn.add_metabolites({model.metabolites.get_by_id(met):-1*float(df_biomass[df_biomass["Unnamed: 0"]==met]["leaf"])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
model.add_reaction(rxn)

rxn = Reaction("Biomass_stem_tx")
for met in df_biomass["Unnamed: 0"]:
    rxn.add_metabolites({model.metabolites.get_by_id(met):-1*float(df_biomass[df_biomass["Unnamed: 0"]==met]["stem"])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
model.add_reaction(rxn)

rxn = Reaction("Biomass_root_tx")
for met in df_biomass["Unnamed: 0"]:
    rxn.add_metabolites({model.metabolites.get_by_id(met):-1*float(df_biomass[df_biomass["Unnamed: 0"]==met]["root"])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
model.add_reaction(rxn)

rxn = Reaction("Biomass_seed_tx")
for met in df_biomass["Unnamed: 0"]:
    rxn.add_metabolites({model.metabolites.get_by_id(met):-1*float(df_biomass[df_biomass["Unnamed: 0"]==met]["seed"])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
model.add_reaction(rxn)

KeyError: 'CPD_17291'

### Run stem

In [ ]:
from cobra import flux_analysis
stem_model = model.copy()
stem_model.reactions.GLC_tx.upper_bound = 0
stem_model.reactions.GLC_tx.lower_bound = 0
stem_model.reactions.Sucrose_tx.upper_bound = 1
stem_model.reactions.Sucrose_tx.lower_bound = 1
stem_model.reactions.Photon_tx.upper_bound = 0
stem_model.reactions.Photon_tx.lower_bound = 0
stem_model.reactions.Biomass_stem_tx.objective_coefficient=1
flux_analysis.parsimonious.pfba(stem_model)
print("Biomass flux ="+str(stem_model.reactions.Biomass_stem_tx.flux))

### Run root

In [ ]:
from cobra import flux_analysis
root_model = model.copy()
root_model.reactions.GLC_tx.upper_bound = 0
root_model.reactions.GLC_tx.lower_bound = 0
root_model.reactions.Sucrose_tx.upper_bound = 1
root_model.reactions.Sucrose_tx.lower_bound = 1
root_model.reactions.Photon_tx.upper_bound = 0
root_model.reactions.Photon_tx.lower_bound = 0
root_model.reactions.Biomass_root_tx.objective_coefficient=1
flux_analysis.parsimonious.pfba(root_model)
print("Biomass flux ="+str(root_model.reactions.Biomass_root_tx.flux))

### Run seed

In [ ]:
from cobra import flux_analysis
seed_model = model.copy()
seed_model.reactions.GLC_tx.upper_bound = 0
seed_model.reactions.GLC_tx.lower_bound = 0
seed_model.reactions.Sucrose_tx.upper_bound = 1
seed_model.reactions.Sucrose_tx.lower_bound = 1
seed_model.reactions.Photon_tx.upper_bound = 0
seed_model.reactions.Photon_tx.lower_bound = 0
seed_model.reactions.Biomass_seed_tx.objective_coefficient=1
sol=flux_analysis.parsimonious.pfba(seed_model)
print("Biomass flux ="+str(seed_model.reactions.Biomass_seed_tx.flux))

    
    

### Generate diel leaf model

In [15]:
########################################################
#This function was used to set up a C3 leaf diel model #
########################################################
def setupC3DielModel(core_model,transferMets,starch_sucrose_ratio=90.0/100):
    from cobra.core import Metabolite, Reaction
    import re

    #create two copies of model elements for day and night
    cobra_model2 = core_model.copy()
    for met in cobra_model2.metabolites:
        met.id = met.id+"1"
        met.compartment = met.compartment+"1"
    for rxn in cobra_model2.reactions:
        rxn.id = rxn.id+"1"

    cobra_model3 = core_model.copy()
    for met in cobra_model3.metabolites:
        met.id = met.id+"2"
        met.compartment = met.compartment+"2"
    for rxn in cobra_model3.reactions:
        rxn.id = rxn.id+"2"

    #merge the day and night model
    cobra_model = cobra_model2+cobra_model3
    for met in cobra_model3.metabolites:
        if not cobra_model.metabolites.__contains__(met.id):
            cobra_model.add_metabolites(met.copy())

    met1 = Metabolite("X_Phloem_contribution_t1",name="Phloem output during the day")
    cobra_model.reactions.get_by_id("Phloem_output_tx1").add_metabolites({met1:1})
    met2 = Metabolite("X_Phloem_contribution_t2",name="Phloem output during at night")
    cobra_model.reactions.get_by_id("Phloem_output_tx2").add_metabolites({met2:1})

    rxn = Reaction("diel_biomass")
    rxn.add_metabolites({met1:-3,met2:-1})
    rxn.lower_bound = 0
    rxn.upper_bound = 1000
    cobra_model.add_reaction(rxn)

    #Adding reactions to allow for day-night metabolite accumulations
    tmfile = open(transferMets,"r")
    tmset=set()
    for line in tmfile:
        tmset.add(line.replace("\n",""))

    for met in tmset:
        if met == "AMMONIUM_v" or met=="FRUCTAN_v":
            continue
        tempRxn = Reaction(met+"_dielTransfer")
        tempRxn.add_metabolites({cobra_model.metabolites.get_by_id(met+"1"):-1,cobra_model.metabolites.get_by_id(met+"2"):1})
        tempRxn.lower_bound=-1000
        if not ((met == "STARCH_p") or (met == "SUCROSE_v") or (met == "MAL_v") or (met == "aMAL_v") or (met == "NITRATE_v") or (met == "CIT_v") or (met == "aCIT_v") or (met == "PROTON_v")):
            tempRxn.lower_bound=0
        tempRxn.upper_bound=1000
        cobra_model.add_reaction(tempRxn)

    fractionMets=dict()
    for rxn in cobra_model.reactions:
        for met in rxn.metabolites.keys():
            prefix=""
            a=re.search("^a{1,3}",met.id)
            anion=""
            if a:
                anion=a.group(0)
                prefix=anion
            b=re.search("^b{1,3}",met.id)
            basic=""
            if b:
                basic=b.group(0)
                prefix=basic
            if ((not prefix == "") and met.compartment == "v1"):
                fractionMets[met]=prefix

    temp=cobra_model.copy()
    for met in fractionMets.keys():
        for rxn in met.reactions:
            if rxn.id.__contains__("_dielTransfer"):
                continue
            else:
                mainMet = met.id[len(fractionMets[met]):]
                coeff1 = temp.reactions.get_by_id(rxn.id).metabolites.get(temp.metabolites.get_by_id(mainMet))
                coeff2 = temp.reactions.get_by_id(rxn.id).metabolites.get(temp.metabolites.get_by_id(met.id))
                if not coeff1:
                    coeff1=0
                if not coeff2:
                    coeff2=0
                total = coeff1 + coeff2
                coeff1 = float(coeff1)/total
                coeff2 = float(coeff2)/total
                if cobra_model.reactions.has_id(met.id[0:len(met.id)-1]+"_dielTransfer"):
                    ub = temp.reactions.get_by_id(met.id[0:len(met.id)-1]+"_dielTransfer").upper_bound
                    lb = temp.reactions.get_by_id(met.id[0:len(met.id)-1]+"_dielTransfer").lower_bound
                    temp.reactions.get_by_id(met.id[0:len(met.id)-1]+"_dielTransfer").remove_from_model()
                    temp.reactions.get_by_id(mainMet[0:len(mainMet)-1]+"_dielTransfer").remove_from_model()
                    Reac = Reaction(mainMet[0:len(mainMet)-1]+"_dielTransfer",name=mainMet+"_dielTransfer")
                    Reac.add_metabolites({temp.metabolites.get_by_id(met.id[0:len(met.id)-1]+"1"):-coeff2,temp.metabolites.get_by_id(met.id[0:len(met.id)-1]+"2"):coeff2,temp.metabolites.get_by_id(mainMet[0:len(mainMet)-1]+"1"):-coeff1,temp.metabolites.get_by_id(mainMet[0:len(mainMet)-1]+"2"):coeff1})
                    Reac.lower_bound=lb
                    Reac.upper_bound=ub
                    temp.add_reaction(Reac)
                    print Reac.reaction
                break
    ####ADD CONSTRAINTS TO MODEL####
    cobra_model = temp.copy()

    #objective function
    cobra_model.reactions.get_by_id("diel_biomass").objective_coefficient=1
    #Leaves - light
    cobra_model.reactions.get_by_id("Sucrose_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("Sucrose_tx1").upper_bound=0
    cobra_model.reactions.get_by_id("GLC_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("GLC_tx1").upper_bound=0
    cobra_model.reactions.get_by_id("CO2_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("NH4_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("NH4_tx1").upper_bound=0
    #Leaves - dark
    cobra_model.reactions.get_by_id("Sucrose_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("Sucrose_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("GLC_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("GLC_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("Photon_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("Photon_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("NH4_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("NH4_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("CO2_tx2").upper_bound=0

    #Set pG6P transporter to 0
    cobra_model.reactions.get_by_id("G6P_Pi_pc1").lower_bound=0
    cobra_model.reactions.get_by_id("G6P_Pi_pc1").upper_bound=0
    cobra_model.reactions.get_by_id("G6P_Pi_pc2").lower_bound=0
    cobra_model.reactions.get_by_id("G6P_Pi_pc2").upper_bound=0

    #Turn off PTOX
    cobra_model.reactions.get_by_id("Plastoquinol_Oxidase_p1").lower_bound=0
    cobra_model.reactions.get_by_id("Plastoquinol_Oxidase_p1").upper_bound=0

    #nitrate uptake constrain
    Nitrate_balance = Metabolite("Nitrate_bal_c", name = "Weights to balance nitrate uptake", compartment = "c1")
    cobra_model.reactions.get_by_id("Nitrate_ec1").add_metabolites({Nitrate_balance:-2})
    cobra_model.reactions.get_by_id("Nitrate_ec2").add_metabolites({Nitrate_balance:3})

    #Rubisco balance
    Rubisco_balance = Metabolite("rubisco_bal_p1", name = "Weights to balance RuBP carboxygenase oxygenase balance", compartment = "p1")
    cobra_model.reactions.get_by_id("RXN_961_p1").add_metabolites({Rubisco_balance:3})
    cobra_model.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1").add_metabolites({Rubisco_balance:-1})

    #generic ATPase and NADPH oxidase
    Maintenance_constraint = Metabolite("ATPase_NADPHoxidase_constraint_c1",name =  "ATPase_NADPHoxidase_constraint_c1", compartment = "c1")
    Maintenance_constraint2 = Metabolite("ATPase_NADPHoxidase_constraint_c2",name =  "ATPase_NADPHoxidase_constraint_c2", compartment = "c2")
    Maintenance_constraint3 = Metabolite("Light_dark_maintainence_constraint",name =  "Light_dark_maintainence_constraint", compartment = "c1")
    cobra_model.reactions.get_by_id("ATPase_tx1").add_metabolites({Maintenance_constraint:1,Maintenance_constraint3:1})
    cobra_model.reactions.get_by_id("ATPase_tx2").add_metabolites({Maintenance_constraint2:1,Maintenance_constraint3:-1})
    cobra_model.reactions.get_by_id("NADPHoxc_tx1").add_metabolites({Maintenance_constraint:-3})
    cobra_model.reactions.get_by_id("NADPHoxc_tx2").add_metabolites({Maintenance_constraint2:-3})
    cobra_model.reactions.get_by_id("NADPHoxm_tx1").add_metabolites({Maintenance_constraint:-3})
    cobra_model.reactions.get_by_id("NADPHoxm_tx2").add_metabolites({Maintenance_constraint2:-3})
    cobra_model.reactions.get_by_id("NADPHoxp_tx1").add_metabolites({Maintenance_constraint:-3})
    cobra_model.reactions.get_by_id("NADPHoxp_tx2").add_metabolites({Maintenance_constraint2:-3})

    ##constrain sucrose and starch storage
    Sucorse_starch_balance = Metabolite("sucrose_starch_bal_c", name = "Weights to balance sucrose-starch uptake", compartment = "c1")
    cobra_model.reactions.get_by_id("SUCROSE_v_dielTransfer").add_metabolites({Sucorse_starch_balance:-1*starch_sucrose_ratio})
    cobra_model.reactions.get_by_id("STARCH_p_dielTransfer").add_metabolites({Sucorse_starch_balance:1})

    #Plastid enolase was not detected in Arabidopsis mesophyll tissue
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p1").lower_bound=0
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p1").upper_bound=0
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p2").lower_bound=0
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p2").upper_bound=0

    #Setting chloroplastic NADPH dehydrogenase to 0  ((Yamamoto et al., 2011)
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p1").lower_bound=0
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p1").upper_bound=0
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p2").lower_bound=0
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p2").upper_bound=0

    #ATP_ADP_Pi constrained to 0 because while there is evidence for its existance, it does not carry flux during the day
    cobra_model.reactions.get_by_id("ATP_ADP_Pi_pc1").lower_bound = 0
    cobra_model.reactions.get_by_id("ATP_ADP_Pi_pc1").upper_bound = 0

    return cobra_model


In [16]:
leaf_model = model.copy()

leaf_model = setupC3DielModel(leaf_model,"MetabolitesToTransfer.txt",starch_sucrose_ratio=5)

for rxn in leaf_model.reactions:
    if rxn.lower_bound==-1000:
        rxn.lower_bound=-1500
    if rxn.upper_bound==1000:
        rxn.upper_bound=1500


PPFD = 1500
ATPase = (0.0049*PPFD) + 2.7851
leaf_model.reactions.get_by_id("ATPase_tx1").lower_bound = ATPase
leaf_model.reactions.get_by_id("ATPase_tx1").upper_bound = ATPase
leaf_model.reactions.diel_biomass.objective_coefficient=1
sol = flux_analysis.parsimonious.pfba(leaf_model)
print(leaf_model.reactions.CO2_tx1.flux)

from sweetlovegroup import analysis
temp = leaf_model.copy()
out_400 = analysis.estimateOutputFromNetCO2(temp,21.645569620253173)
leaf_model_400 = leaf_model.copy()
leaf_model_400.reactions.diel_biomass.lower_bound = out_400
leaf_model_400.reactions.diel_biomass.upper_bound = out_400
sol = flux_analysis.parsimonious.pfba(leaf_model_400)
print("Phloem output = ")
print(leaf_model_400.reactions.diel_biomass.flux)


temp = leaf_model.copy()
out_550 = analysis.estimateOutputFromNetCO2(temp,24.43037974683546)
leaf_model_550 = leaf_model.copy()
leaf_model_550.reactions.diel_biomass.lower_bound = out_550
leaf_model_550.reactions.diel_biomass.upper_bound = out_550
sol = flux_analysis.parsimonious.pfba(leaf_model_550)
print("Phloem output = ")
print(leaf_model_550.reactions.diel_biomass.flux)

0.5 CIT_v1 + 0.5 aCIT_v1 <=> 0.5 CIT_v2 + 0.5 aCIT_v2
0.7 MAL_v1 + 0.3 aMAL_v1 <=> 0.7 MAL_v2 + 0.3 aMAL_v2
bHIS_v1 --> bHIS_v2
106.674498326
1500
10.1351
21.6455696203
Phloem output = 
0.452844044741
1500
10.1351
24.4303797468
Phloem output = 
0.521458572285


In [39]:
leaf_model.compartments={'b1': 'Biomass_1',"b2":"Biomass_2"}

In [41]:
for comp in leaf_model.compartments.keys():
    print comp
    print leaf_model.compartments[comp]

b1
Biomass_1
b2
Biomass_2
mc2

mc1

m1

m2

i1

i2

v1

v2

x2

c2

c1

x1

e1

e2

p2

p1

r1

r2

l2

l1

mi1

mi2



In [18]:
ActiveRxnNotInTranscriptomicsData=set()
for rxn in leaf_model.reactions:
    if abs(rxn.flux) > 0 and len(rxn.genes)!=0:
        flag=False
        for gene in rxn.genes:
            if gene.id in set(df["Glyma2.0"]):
                flag=True
                print "Gene "+gene.id+"for rxn "+rxn.id+" present"
                break
        if not flag:
            ActiveRxnNotInTranscriptomicsData.add(rxn.id)

Gene Glyma.06G074900for rxn ACETYLGLUTKIN_RXN_p1 present
Gene Glyma.13G140700for rxn DIAMINOPIMDECARB_RXN_p1 present
Gene Glyma.02G222400for rxn F16ALDOLASE_RXN_p1 present
Gene Glyma.02G222400for rxn F16ALDOLASE_RXN_c1 present
Gene Glyma.20G079400for rxn NADH_DEHYDROG_A_RXN_mi1 present
Gene Glyma.13G097100for rxn RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1 present
Gene Glyma.14G053300for rxn IGPSYN_RXN_p1 present
Gene Glyma.10G016200for rxn FUMHYDR_RXN_m1 present
Gene Glyma.13G204700for rxn GLYCOGENSYN_RXN_p1 present
Gene Glyma.01G026700for rxn ALANINE_AMINOTRANSFERASE_RXN_c1 present
Gene Glyma.03G173400for rxn DIAMINOPIMEPIM_RXN_p1 present
Gene Glyma.10G064000for rxn THREDEHYD_RXN_p1 present
Gene Glyma.20G148100for rxn ACSERLY_RXN_c1 present
Gene Glyma.17G159700for rxn 2_ISOPROPYLMALATESYN_RXN_p1 present
Gene Glyma.14G217300for rxn 3_ISOPROPYLMALISOM_RXN_p1 present
Gene Glyma.12G042400for rxn GLUC1PADENYLTRANS_RXN_p1 present
Gene Glyma.20G148100for rxn ACSERLY_RXN_p1 present
Gene Glyma.1

Gene Glyma.09G167000for rxn SUCROSE_SYNTHASE_RXN_c1 present
Gene Glyma.10G211400for rxn CHORISMATE_SYNTHASE_RXN_p1 present
Gene Glyma.03G129700for rxn CYSTATHIONINE_BETA_LYASE_RXN_p1 present
Gene Glyma.13G153700for rxn PRIBFAICARPISOM_RXN_p1 present
Gene Glyma.03G212700for rxn METHYLENETHFDEHYDROG_NADP_RXN_c1 present
Gene Glyma.02G222400for rxn F16ALDOLASE_RXN_p2 present
Gene Glyma.02G222400for rxn F16ALDOLASE_RXN_c2 present
Gene Glyma.20G079400for rxn NADH_DEHYDROG_A_RXN_mi2 present
Gene Glyma.11G101400for rxn RXN_9663_p2 present
Gene Glyma.02G015700for rxn FUMHYDR_RXN_m2 present
Gene Glyma.02G015700for rxn FUMHYDR_RXN_c2 present
Gene Glyma.11G248700for rxn UREASE_RXN_c2 present
Gene Glyma.01G026700for rxn ALANINE_AMINOTRANSFERASE_RXN_c2 present
Gene Glyma.17G159700for rxn 2_ISOPROPYLMALATESYN_RXN_p2 present
Gene Glyma.14G217300for rxn 3_ISOPROPYLMALISOM_RXN_p2 present
Gene Glyma.20G148100for rxn ACSERLY_RXN_p2 present
Gene Glyma.11G101400for rxn RXN_9661_p2 present
Gene Glyma.12G1938

In [19]:
ActiveRxnNotInTranscriptomicsData

{'1_ACYLGLYCEROL_3_P_ACYLTRANSFER_RXN_p2',
 'GLUCOKIN_RXN_c2',
 'GLY3KIN_RXN_p1',
 'PHOSGLYPHOS_RXN_c1',
 'PHOSGLYPHOS_RXN_c2',
 'PHOSGLYPHOS_RXN_p1',
 'PHOSGLYPHOS_RXN_p2',
 'THREONINE_ALDOLASE_RXN_c1'}

In [20]:
df_amb[df_amb["Glyma2.0"]=="Glyma.14G053300"][df_amb.columns[17:]].mean().mean()

6.7630348276875

In [21]:
totalTranscript_400 = dict()
for rxn in leaf_model.reactions:
    x=0
    for gene in rxn.genes:
        #print gene
        y = df_amb[df_amb["Glyma2.0"]==gene.id][df_amb.columns[17:]].mean().mean()
        #print y
        if pd.isna(y):
            x = x + 0
        else:
            x = x + y
    totalTranscript_400[rxn.id]=x
    
totalTranscript_550 = dict()
for rxn in leaf_model.reactions:
    x=0
    for gene in rxn.genes:
        #print gene
        y = df_ele[df_ele["Glyma2.0"]==gene.id][df_ele.columns[17:]].mean().mean()
        #print y
        if pd.isna(y):
            x = x + 0
        else:
            x = x + y
    totalTranscript_550[rxn.id]=x

In [22]:
rxnList = list()
transcriptList = list()
fluxList = list()
import numpy as np

for rxnID in totalTranscript_550.keys():
    rxnList.append(rxnID)
    a=0
    b=0
    if round(totalTranscript_400[rxnID],3)==0 and round(totalTranscript_550[rxnID],3)==0:
        transcriptList.append(0)
    elif round(totalTranscript_400[rxnID],3)==0:
        transcriptList.append(np.nan)
    else:
        a = totalTranscript_550[rxnID]/totalTranscript_400[rxnID]
        transcriptList.append(a)
    if round(leaf_model_550.reactions.get_by_id(rxnID).flux,3)==0 and round(leaf_model_400.reactions.get_by_id(rxnID).flux,3)==0:
        fluxList.append(0)
    elif round(leaf_model_400.reactions.get_by_id(rxnID).flux,3)==0:
        fluxList.append(np.nan)
    else:
        b = leaf_model_550.reactions.get_by_id(rxnID).flux/leaf_model_400.reactions.get_by_id(rxnID).flux
        fluxList.append(b)
    if not (a==0 and b==0):
        if b == 0:
            continue
        if round(a/b,2)>0.98 and round(a/b,2)<1.02:
            print rxnID
            print a
            print b
            print a/b

df_ratios = pd.DataFrame(data={"rxn ID":rxnList,"Transcript ratio":transcriptList,"Flux ratio":fluxList})

INORGPYROPHOSPHAT_RXN_p1
1.101657328803641
1.08903172312
1.0115934232333557
NADH_DEHYDROG_A_RXN_mi2
1.0046849867090821
1.01675560787
0.9881282964439743
GAPOXNPHOSPHN_RXN_c2
1.0193884551113048
1.01191025142
1.007390184735083
PEPDEPHOS_RXN_c2
1.0215797817779837
1.01178551139
1.0096801844678556
FUMHYDR_RXN_m2
0.9982047427477271
1.00721935887
0.9910499971563462
TRIOSEPISOMERIZATION_RXN_p1
1.0072391861039773
1.01857028924
0.9888754823771729
RIB5PISOM_RXN_p2
0.979800200721292
0.994290325557
0.9854266661725176
PHOSPHOGLUCMUT_RXN_p2
1.0003436901517928
0.994290325557
1.0060881258110699
CITSYN_RXN_m2
1.0367447835558694
1.02202578301
1.0144017898517175
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi2
1.0057316875838567
1.00717613236
0.9985658468948113
TRANSALDOL_RXN_p2
1.0012607681881907
0.994223426223
1.0070782298827117
MALATE_DEH_RXN_c2
1.0152293808837947
1.01191025142
1.0032800630897292
MALATE_DEH_RXN_m2
1.0152293808837947
1.01737953002
0.9978865810950349
SUCCCOASYN_RXN_m2
1.010030296025145
1.007176

In [23]:
totalTranscript_400["INORGPYROPHOSPHAT_RXN_p1"]

111.53797876447916

In [ ]:
leaf_model.reactions.L_CITRULLINE_pc1.genes

In [ ]:
leaf_model_sink = model.copy()

leaf_model_sink = setupC3DielModel(leaf_model_sink,"MetabolitesToTransfer.txt",starch_sucrose_ratio=0.6)

met = Metabolite("X_Biomass_contribution_t1")
leaf_model_sink.reactions.Biomass_leaf_tx1.add_metabolites({met:1})
leaf_model_sink.metabolites.X_Phloem_contribution_t1.remove_from_model()
leaf_model_sink.reactions.diel_biomass.add_metabolites({met:-3})

met = Metabolite("X_Biomass_contribution_t2")
leaf_model_sink.reactions.Biomass_leaf_tx2.add_metabolites({met:1})
leaf_model_sink.metabolites.X_Phloem_contribution_t2.remove_from_model()
leaf_model_sink.reactions.diel_biomass.add_metabolites({met:-1})

leaf_model_sink.reactions.diel_biomass.objective_coefficient=1
flux_analysis.parsimonious.pfba(leaf_model_sink)
print("Biomass accumulation = ")
print(leaf_model_sink.reactions.diel_biomass.flux)